In [1]:
import numpy as np
import pandas as pd

import re
import string
import time
from datetime import date, timedelta, datetime

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import yfinance as yf

In [2]:
stock = yf.Ticker('AAPL').history(start='2020-07-01',end=datetime.today())

In [4]:
df = pd.read_csv('data/aapl_articles.csv')
df.set_index('Unnamed: 0',inplace=True)
df.index.rename('adjusted_date',inplace=True)
df['new_date'] = pd.to_datetime(df['new_date'])
df['adjusted_dates'] = [i - timedelta(days=1) if j == 5 else i - timedelta(days=2) if j == 6 else i for i,j in zip(df['new_date'],df['day'])]
fixed_df = df.drop(columns=['Open','High','Low','Close','Volume','Dividends','Stock Splits']).set_index('adjusted_dates')
fixed_df.sort_index(inplace=True)
prepared_df = fixed_df.join(stock.rolling(5).mean().shift(-5).dropna().drop(columns=['Dividends','Stock Splits','Volume']).pct_change(),how='inner').join(stock.rolling(5).mean().shift(-5).dropna()[['Volume']],how='inner')

In [5]:
prepared_df

,date,title,text,new_date,day,Open,High,Low,Close,Volume
2020-07-31,"Aug. 1, 2020 6:21 PM ET",Apple Amazon Microsoft Google and Facebook con...,Five stocks Microsoft Apple Amazon Facebook ...,2020-08-01,5,0.019498,0.013499,0.018043,0.009200,200737840.0
2020-07-31,"Aug. 1, 2020 3:53 PM ET",Apple purges 30000 apps from its China store,Apple NASDAQAAPL has removed more than 30000 ...,2020-08-01,5,0.019498,0.013499,0.018043,0.009200,200737840.0
2020-07-31,"Aug. 1, 2020 10:09 AM ET",India makes big push for smartphone production,Samsung OTCSSNLF Foxconn OTCFXCOF Wistron Cor...,2020-08-01,5,0.019498,0.013499,0.018043,0.009200,200737840.0
2020-07-31,"Jul. 31, 2020 12:52 PM ET",Facebook surge leads tech megacaps higher but ...,Investors continued to reward most megacap te...,2020-07-31,4,0.019498,0.013499,0.018043,0.009200,200737840.0
2020-08-03,"Aug. 3, 2020 4:02 PM ET",Microsoft and Apple take Nasdaq to another all...,The market day started and ended with megacap...,2020-08-03,0,0.008352,0.004178,0.004224,0.007217,181588320.0
...,...,...,...,...,...,...,...,...,...,...
2020-09-15,"Sep. 15, 2020 1:29 PM ET",Apple launches $399 Watch Series 6 $279 Watch ...,IPad Air post update The latest iPad Air has ...,2020-09-15,1,-0.010121,-0.010509,-0.008187,-0.006722,199712820.0
2020-09-16,"Sep. 16, 2020 2:23 PM ET",Retail investors had de minimis impact on late...,If you look at the stocks retail investors we...,2020-09-16,2,-0.006533,-0.006920,-0.009776,-0.009089,198920760.0
2020-09-17,"Sep. 17, 2020 11:00 AM ET",Jefferies raises Apple target on 5G enthusiasm,Jefferies has raised its price target on Appl...,2020-09-17,3,-0.008288,-0.003493,-0.006950,-0.003881,196867220.0
2020-09-18,"Sep. 19, 2020 10:20 AM ET",Apples upsell mastery noted to be on display a...,Loup Ventures Gene Munster sizes up the Apple...,2020-09-19,5,-0.003619,0.002804,0.002980,0.009999,169393500.0


In [100]:
prepared_df.to_csv()

FileNotFoundError: [Errno 2] No such file or directory: '/data/modeling_aapl.csv'